In [ ]:
# Import libraries
import sys
sys.path.append('src/')
from src.model import build_segformer3d_model, SegFormer3D
from src.preprocessing import get_transforms, get_datasets, get_dataloaders
import torch
import matplotlib.pyplot as plt
from src.train_validate import train, validate
from tqdm import tqdm

# Set the device
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

In [ ]:
# Get transforms
transform = get_transforms(resize_shape=[240, 240, 155], contrast_value=1000)

# Get datasets
train_ds, val_ds = get_datasets(
    root_dir="../data",
    collection="HCC-TACE-Seg",
    transform=transform,
    download_len=2,
    val_frac=0.2,
)

# Get dataloaders
train_loader, val_loader = get_dataloaders(train_ds, val_ds, batch_size=2)

# Get datalists
train_dl = train_ds.datalist
val_dl = val_ds.datalist

# Check length of datasets, dataloaders and datalists
print(train_ds.get_indices(), val_ds.get_indices())
print(len(train_loader), len(val_loader))
print(len(train_dl), len(val_dl))

In [ ]:
for inputs in tqdm(train_loader):
    images, targets = inputs["image"], inputs["seg"]

In [ ]:
images, targets = images.to(device), targets.to(device)
model = SegFormer3D()
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.AdamW(model.parameters(), lr=1e-4)

In [ ]:
model = model.to(device)
model.train()
# optimizer.zero_grad()

output = model(images)
print(output.shape)
# loss = criterion(outputs, targets)
# loss.backward()
# optimizer.step()

# print(loss.item())